# Imports

In [1]:
import numpy as np
from pprint import pprint as pp
import re #Used to read chunks from the data files - clusters, vmat, etec.
import math
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import random
from IPython.display import clear_output

from scipy.optimize import minimize, basinhopping
from scipy.optimize import SR1, BFGS
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint

from ase import Atoms
from ase.units import kB
from ase.visualize import view
from ase.build import bulk
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.vasp import write_vasp

import sys
import subprocess
import os

pattern1 = re.compile("\n\n\n")
pattern2 = re.compile("\n\n")
np.set_printoptions(suppress=True,precision=2)

# Read all necessary files

## Read Clusters.out

In [2]:
# Read clusters.out
clusters = {}

with open('clusters.out','r') as fclusters:
    temp_clusters = fclusters.read().split('\n\n') #Read blocks separated by 1 empty line

for idx, cluster in enumerate(temp_clusters):
    if cluster == '': #Check for spurious empty blocks
        continue 
    line = cluster.split('\n') #If not empty split by lines
    multiplicity = int(line[0]) #1st line
    length = float(line[1]) #largest distance between two atoms
    num_points = int(line[2]) #type of cluster
    clusters[idx] = {'mult':multiplicity, 'length':length, 'type':num_points}

In [3]:
clusters

{0: {'mult': 1, 'length': 0.0, 'type': 0},
 1: {'mult': 1, 'length': 0.0, 'type': 1},
 2: {'mult': 4, 'length': 0.86603, 'type': 2},
 3: {'mult': 3, 'length': 1.0, 'type': 2},
 4: {'mult': 12, 'length': 1.0, 'type': 3},
 5: {'mult': 6, 'length': 1.0, 'type': 4}}

## Read Config.out

In [6]:
# Read config.out
configs = {}

fconfig = open('config.out','r')
_ = next(fconfig) #Ignore first line

temp_config = fconfig.read()#.split('\n\n')
temp_config = pattern1.split(temp_config) #split lines separated by 2 empty lines

for idx, config in enumerate(temp_config):
    if config == '': #Check for spurious empty blocks
        continue
    num_points = int(config[0]) #number of subclusters
    config = pattern2.split(config[2:]) #now split individual subclusters separated by 1 blank line
    min_coords = []
    for _ in range(num_points):
        min_coords.append(config[_].split('\n')[0])
    configs[idx] = {'subclus': list(map(int,min_coords)), 'num_of_subclus': len(min_coords)}

In [7]:
configs

{0: {'subclus': [0], 'num_of_subclus': 1},
 1: {'subclus': [1, 1], 'num_of_subclus': 2},
 2: {'subclus': [2, 2, 2], 'num_of_subclus': 3},
 3: {'subclus': [2, 2, 2], 'num_of_subclus': 3},
 4: {'subclus': [3, 3, 3, 3, 3, 3], 'num_of_subclus': 6},
 5: {'subclus': [4, 4, 4, 4, 4, 4], 'num_of_subclus': 6}}

## Read Kikuchi-Barker Coefficients

In [8]:
# Read kb.out

kb = {}
fkb = open('kb.out','r')
_ = next(fkb) #ignore first line

temp_kb = fkb.read()
temp_kb = temp_kb.split('\n') #split file linewise

for idx, kbcoeff in enumerate(temp_kb):
    if kbcoeff == '': #check for spurious empty blocks
        continue
    kb[idx] = float(kbcoeff)

fkb.close()

In [9]:
kb

{0: 0.0, 1: -1.0, 2: 1.0, 3: 1.0, 4: -1.0, 5: 1.0}

## Read Coefficient of subclusters

In [12]:
# Read configcoeff.out
configcoef = {}

with open('configcoef.out','r') as fsubmult:
    _ = next(fsubmult) #ignore first line
    temp_submult = fsubmult.read() 
    temp_submult = pattern2.split(temp_submult) #split lines into blocks separated by 2 empty lines
    
for idx, submult in enumerate(temp_submult):
    submult = submult.split('\n') #split into number of subclusters 
    while("" in submult) :
        submult.remove("") #remove empty blocks
    configcoef[idx] = list(map(float,submult[1:])) #also ignore 1st line of each block

In [13]:
configcoef

{0: [1.0],
 1: [1.0, 1.0],
 2: [1.0, 2.0, 1.0],
 3: [1.0, 2.0, 1.0],
 4: [1.0, 2.0, 1.0, 1.0, 2.0, 1.0],
 5: [1.0, 4.0, 2.0, 4.0, 4.0, 1.0],
 6: [0.0, -1.0, 1.0, 1.0, -1.0, 1.0]}

In [14]:
# Read vmat.out
vmat = {}
with open('vmat.out') as fvmat:
    _ = next(fvmat) #ignore first lie
    temp_vmat = fvmat.read()
    temp_vmat = pattern2.split(temp_vmat) #split by 2 empty lines i.e. maxclusters
    
    while("" in temp_vmat) :
        temp_vmat.remove("") #remove empty blocks
    
    for clus_idx, mat in enumerate(temp_vmat):
        mat = mat.split('\n') #split by 1 empty line i.e. subclusters
        mat_float = np.empty(list(map(int, mat[0].split(' '))))
        for idx, row in enumerate(mat[1:]): #ignore first line
            mat_float[idx] = list(map(float,row.split(' ')[:-1]))
        
        vmat[clus_idx] = mat_float

In [15]:
vmat

{0: array([[1., 0., 0., 0., 0., 0.]]),
 1: array([[ 0.5, -0.5,  0. ,  0. ,  0. ,  0. ],
        [ 0.5,  0.5,  0. ,  0. ,  0. ,  0. ]]),
 2: array([[ 0.25, -0.5 ,  0.25,  0.  ,  0.  ,  0.  ],
        [ 0.25,  0.  , -0.25,  0.  ,  0.  ,  0.  ],
        [ 0.25,  0.5 ,  0.25,  0.  ,  0.  ,  0.  ]]),
 3: array([[ 0.25, -0.5 ,  0.  ,  0.25,  0.  ,  0.  ],
        [ 0.25,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
        [ 0.25,  0.5 ,  0.  ,  0.25,  0.  ,  0.  ]]),
 4: array([[ 0.12, -0.38,  0.25,  0.12, -0.12,  0.  ],
        [ 0.12, -0.12,  0.  , -0.12,  0.12,  0.  ],
        [ 0.12,  0.12, -0.25,  0.12, -0.12,  0.  ],
        [ 0.12, -0.12, -0.25,  0.12,  0.12,  0.  ],
        [ 0.12,  0.12,  0.  , -0.12, -0.12,  0.  ],
        [ 0.12,  0.38,  0.25,  0.12,  0.12,  0.  ]]),
 5: array([[ 0.06, -0.25,  0.25,  0.12, -0.25,  0.06],
        [ 0.06, -0.12,  0.  ,  0.  ,  0.12, -0.06],
        [ 0.06,  0.  , -0.25,  0.12,  0.  ,  0.06],
        [ 0.06,  0.  ,  0.  , -0.12,  0.  ,  0.06],
        [ 0.06

## Read ECI

In [17]:
# Read eci
eci = {}

with open('eci.out') as feci:
    _ = next(feci) #Ignore first line
    temp_eci = feci.read()
    temp_eci = temp_eci.split('\n') #split by line

for idx, eci_val in enumerate(temp_eci):
    if eci_val == '':
        continue
    eci[idx] = float(eci_val)

In [18]:
#In this notebook ECI's are declared manually
eci = {0: 0, 1: 0, 2: 0.5, 3: 0.0, 4: 0, 5: 0}

## Declaring test Correlations

In [19]:
#corrs = np.array([1.    , 0.0   , 0.25,0.25  , 0.125 , 0.0625])
#corrs = np.array([ 1.0, 0,  0.08,  0.23,-0.00054,  0.165])
#corrs = np.array([ 1.  , 0.0 ,  0.25 ,  -0.25 ,  0.125 ,  0.0])
corrs1 = np.array([1., 1., 1., 1., 1., 1.]) #Pure B
corrs0 = np.array([1., -1., 1., 1., -1., 1.]) #Pure A
corrsrand = np.array([1.    , 0.0   , 0.25,0.25  , 0.125 , 0.0625]) # AB - sqs
#np.array([1.0, 0.0, -0.01852, -0.1358, -0.01235, -0.06173])
#np.array([ 1.,    0.,    0.04,  0.14, -0.07,  0.12])
#np.array([1.    , 0.0   , 0.25,0.25  , 0.125 , 0.0625])

# Optimisation

## Setting up F, F Jacobian and F Hessian

### Setup test temperature

In [23]:
T = 1/kB

### Old Declaration (Kept for backup, Ignore)

In [21]:
def F_old(corrs, vmat, kb, clusters, configs, configcoef,T,eci):

    S = 0
    H = 0
    
    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)

        return rho
    
    def inner_sum(cluster_idx):
        isum = 0
        rho = clus_prob(cluster_idx)
        #print(rho)
        for i in range(configs[cluster_idx]['num_of_subclus']):
            try:
                if rho[i] == 0:
                    isum += configcoef[cluster_idx][i] * rho[i] * math.log(np.finfo(float).eps)
                else:
                    isum += configcoef[cluster_idx][i] * rho[i] * math.log(rho[i])
            except ValueError as ve:
                print('Math Domain Error. Check the validity of the correlations')
                print(f'corrs: {corrs} \n Correlation Sum: {rho[i]}')
                pass
        return isum 
            
    for cluster_idx, cluster in clusters.items():
        H += cluster['mult']*eci[cluster_idx]*corrs[cluster_idx]
        S += kb[cluster_idx]*inner_sum(cluster_idx)
        
    #return kB*S
    return H + kB*T*S

In [24]:
f_old0 = F_old(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_old1 = F_old(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_rand = F_old(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f_old0:.2f} -- Corrs 1: {f_old1:.2f} -- Corrs Rand: {f_rand:.2f}")

Corrs 0: 2.00 -- Corrs 1: 2.00 -- Corrs Rand: -2.13


### Free Energy Function

In [26]:
def F(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    F = H + T*SUM(rho * log(rho))
    """
    
    H = 0
    S = 0
    
    def get_corr_sum(corrs,vmat,cluster_index,config_idx):
        """
        returns rho for a particular subcluster of a maxcluster
        """
        #TODO: This loop can be reduced as a Dot Product
        corrsum = 0
        for corr_idx, corr in enumerate(corrs):
            corrsum += vmat[cluster_idx][config_idx][corr_idx]*corr

        return corrsum
    
    #Loop through all maximal clusters
    for cluster_idx, cluster in clusters.items():
        
        H += cluster['mult']*eci[cluster_idx]*corrs[cluster_idx] #H is a func of maxclux only
        
        temp_S = 0
        
        #Loop through subclusters of each maximal cluster
        for config_idx, config in enumerate(configcoef[cluster_idx]):
            corrsum = get_corr_sum(corrs,vmat,cluster_idx,config_idx)
            try:
                if corrsum == 0:
                    #If rho is 0, then replace with smallest +ve number as per machine precision
                    #The log doesn't matter, cuz, although log of a small number is a finite number, the 0multiplicative factor makes it 0 any
                    temp_S += config*corrsum*math.log(np.finfo(float).tiny)
                else:
                    temp_S += config*corrsum*math.log(corrsum)
            except ValueError as ve:
                pass
                #Exception handling retained for possible -ve rhos
                print('Math Domain Error. Check the validity of the correlations')
                print(f'corrs: {corrs} \n Correlation Sum: {corrsum}')
                
        S += kb[cluster_idx]*temp_S
    
    return H + kB*T*S 

In [27]:
f0 = F(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f1 = F(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
frand = F(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f0:.2f} -- Corrs 1: {f1:.2f} -- Corrs Rand: {frand:.2f}")

Corrs 0: 2.00 -- Corrs 1: 2.00 -- Corrs Rand: -2.13


### Free Energy Jacobian

In [34]:
def F_jacobian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [dF/dcorr0, dF/dcorr1, ...]
    """
    
    def get_corr_sum(corrs,vmat,cluster_index,config_idx):
        """
        returns rho for a particular subcluster of a maxcluster
        """
        #TODO: This loop can be reduced as a Dot Product
        corrsum = 0
        for corr_idx, corr in enumerate(corrs):
            corrsum += vmat[cluster_idx][config_idx][corr_idx]*corr
        
        return corrsum
            
    
    F_jac = []
    #Loop through each correlation
    for corr_idx, corr in enumerate(corrs):
        
        temp_S_jac = 0
        
        #Loop through all maxclus, since the rhos depend on the correlation
        for cluster_idx, cluster in clusters.items():
            
            temp_config_sum = 0
            #Loop through all subclusters in a maximal cluster
            for config_idx, config in enumerate(configcoef[cluster_idx]):
                corrsum = get_corr_sum(corrs,vmat,cluster_idx,config_idx)
                try:
                    if corrsum == 0:
                        #temp_config_sum += config*vmat[cluster_idx][config_idx][corr_idx]*(1 + np.log(corrsum))
                        #^^ same line can be used since np.log(x) automaticaly returns -infty if x = 0
                        #Can't use tiny, since it gives a very small value, but log of that value gives a finite value. Note that we took tiny in the F calculation but since it had another multiplicative factor which is also small, the finite log value didn't matter)
                        temp_config_sum += np.NINF #negative infinity
                        
                    else:
                        temp_config_sum += config*vmat[cluster_idx][config_idx][corr_idx]*(1 + np.log(corrsum))
                except ValueError as ve:
                    #Exception handling retained for possible -ve rhos
                    print('Math Domain Error. Check the validity of the correlations')
                    print(f'corrs: {corrs} \n Correlation Sum: {corrsum}')
                    pass
            
            temp_S_jac += kb[cluster_idx]*temp_config_sum
        
        F_jac.append(clusters[corr_idx]['mult']*eci[corr_idx] + kB*T*temp_S_jac)
        
    return np.array(F_jac)

In [35]:
f_jaco0 = F_jacobian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_jaco1 = F_jacobian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_jacorand = F_jacobian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f_jaco0} \n Corrs 1: {f_jaco1} \n Corrs Rand {f_jacorand}")

Corrs 0: [nan nan nan nan nan nan] 
 Corrs 1: [nan nan nan nan nan nan] 
 Corrs Rand [-1.9  -0.1   2.57  0.41  0.23 -0.05]


<ipython-input-34-42d6047e7b4b>:57: RuntimeWarning: invalid value encountered in double_scalars
  temp_S_jac += kb[cluster_idx]*temp_config_sum


### Free Energy Hessian

In [42]:
def F_hessian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [[d^2F/dcorr0 dcorr0, d^2F/dcorr0 dcorr1, ..., d^2F/dcorr0 dcorrn],
     [d^2F/dcorr1 dcorr0, d^2F/dcorr1 dcorr1, ..., d^2F/dcorr1 dcorrn],
     .
     .
     .
     [d^2F/dcorrn dcorr0, d^2F/dcorrn dcorr1, ..., d^2F/dcorrn dcorrn],
    ]
    """
    
    F_hess = np.empty([len(corrs),len(corrs)])
    
    def get_corr_sum(corrs,vmat,cluster_index,config_idx):
        corrsum = 0
        for corr_idx, corr in enumerate(corrs):
            corrsum += vmat[cluster_idx][config_idx][corr_idx]*corr

        return corrsum
    
    #Enuemrate through all possible correlation pairs
    for corr_idx_1, corr_1 in enumerate(corrs):
        for corr_idx_2, corr_2 in enumerate(corrs):
            
            temp_hess = 0
            #loop through maximal clusters
            for cluster_idx, cluster in clusters.items():
                temp_config_sum = 0
                #loop through subclustes for the present maximal cluster
                for config_idx, config in enumerate(configcoef[cluster_idx]):
                    corrsum = get_corr_sum(corrs,vmat,cluster_idx,config_idx)
                    try:
                        if corrsum == 0:
                            temp_config_sum += config*vmat[cluster_idx][config_idx][corr_idx_1]*vmat[cluster_idx][config_idx][corr_idx_2]/np.finfo(float).tiny #dividing by a small number.
                            #Note that you can also set the value as +ve infinity.
                            #temp_config_sum += np.PINF
                        else:
                            temp_config_sum += config*vmat[cluster_idx][config_idx][corr_idx_1]*vmat[cluster_idx][config_idx][corr_idx_2]/corrsum
                    except ValueError as ve:
                        #Exception handling retained for possible -ve rhos
                        print('Math Domain Error. Check the validity of the correlations')
                        print(f'corrs: {corrs} \n Correlation Sum: {corrsum}')
                        pass
                temp_hess += kb[cluster_idx]*temp_config_sum
            F_hess[corr_idx_1][corr_idx_2] = kB*T*temp_hess
    
    return F_hess

In [43]:
f_hess0 = F_hessian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_hess1 = F_hessian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_hessrand = F_hessian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0:\n {f_hess0} \n Corrs 1:\n {f_hess1}\n Corrs Rand:\n {f_hessrand}")

Corrs 0:
 [[ 3.34e+306 -1.40e+306 -2.11e+306 -2.46e+306  0.00e+000 -1.76e+305]
 [-1.40e+306  9.13e+306  4.21e+306  4.92e+306 -7.02e+305  7.02e+305]
 [-2.11e+306  4.21e+306  8.43e+306  0.00e+000  1.40e+306 -7.02e+305]
 [-2.46e+306  4.92e+306  0.00e+000  8.43e+306  7.02e+305 -3.51e+305]
 [ 0.00e+000 -7.02e+305  1.40e+306  7.02e+305  3.51e+306  7.02e+305]
 [-1.76e+305  7.02e+305 -7.02e+305 -3.51e+305  7.02e+305  2.63e+306]] 
 Corrs 1:
 [[ 3.34e+306  1.40e+306 -2.11e+306 -2.46e+306  0.00e+000 -1.76e+305]
 [ 1.40e+306  9.13e+306 -4.21e+306 -4.92e+306 -7.02e+305 -7.02e+305]
 [-2.11e+306 -4.21e+306  8.43e+306  0.00e+000 -1.40e+306 -7.02e+305]
 [-2.46e+306 -4.92e+306  0.00e+000  8.43e+306 -7.02e+305 -3.51e+305]
 [ 0.00e+000 -7.02e+305 -1.40e+306 -7.02e+305  3.51e+306 -7.02e+305]
 [-1.76e+305 -7.02e+305 -7.02e+305 -3.51e+305 -7.02e+305  2.63e+306]]
 Corrs Rand:
 [[ 1.25  0.13 -0.55 -0.41 -0.16  0.14]
 [ 0.13  2.99  0.02  0.01 -0.89 -0.35]
 [-0.55  0.02  3.49 -0.97 -0.25 -0.69]
 [-0.41  0.01 -0.

## Setting up Constraints

In [44]:
def constraint_rhos_sum(corrs, vmat, clusters, configcoef,):
    """
    Constraints the sum of each rho. As of now, it's done in a weird way, where the total sum of the array:
    [1 - sum(rho), .... ] is constrained to sum to 0. This along with the constraint that each rho is between
    0 and 1, seems to make it work. I think that by the this might be a redundant constraint as well.
    """
    rho_sum = []

    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        rho_sum.append(np.sum(configcoef[cluster_idx]*rho))
    
    return np.sum(1 - np.array(rho_sum))

def constraint_singlet(corrs,FIXED_CORR_1):
    """
    constrains the 1-pt correlation:
    corrs[1] = FIXED_CORR_1
    """
    return corrs[1] - FIXED_CORR_1   

def constraint_zero(corrs):
    """
    constrains the 1-pt correlation:
    corrs[0] = 1
    """
    return 1 - corrs[0]  

In [45]:
def get_random_corr(correlation):
    """
    Returns a random set of correlations for a BCC binary alloy with a fixed 1-pt correlation
    """
    concentration = (correlation - (-1))/(1 - (-1)) #scale factor
    a=0.5
    atoms = crystal(['Fe'], [(0,0,0)], spacegroup=229, 
                    cellpar=[1, 1, 1, 90, 90, 90])
    
    scell = make_supercell(atoms,np.eye(3)*3)
    temp_atomic_nums = scell.get_atomic_numbers()
    mask = np.zeros(len(temp_atomic_nums), dtype=int).astype(bool)
    mask[:int(concentration*len(temp_atomic_nums))] = 1.0
    np.random.shuffle(mask)
    temp_atomic_nums[mask] = 13
    scell.set_atomic_numbers(temp_atomic_nums)
    
    write_vasp('POSCAR.temp',scell,sort=True,direct=True)
    _ = subprocess.call('./poscar2str',shell=True)
    
    corrs = subprocess.call('corrdump -c',shell=True)
    
    popen = subprocess.Popen('corrdump -c'.split(), stdout=subprocess.PIPE)
    popen.wait()
    return [float(corr) for corr in popen.stdout.read().decode("utf-8").split('\t')[:-1]]

In [ ]:
get_random_corr(0)

In [ ]:
def basin_hopping_callback(corrs, G, accept):
    """
    Function to print diagnostic while fitting. 
    Not being used right now
    """
    if accept == True:
        clear_output(wait=True)
        print(f'1-pt Correlation: {corrs[1]:.2f}')
        print(f'Concentation: {(corrs[1] - (-1))/(1 - (-1)):.2f}')
        print(f'New Minima found --> G: {G:.2f}')
        print('Current Rho:')
        for cluster_idx in clusters:
            print(np.matmul(vmat[cluster_idx],corrs))
        print("===========================")

In [48]:
np.linspace(0, 3000, num=11), np.linspace(0, 3000, num=11),np.linspace(-1,1,17)

(array([   0.,  300.,  600.,  900., 1200., 1500., 1800., 2100., 2400.,
        2700., 3000.]),
 array([   0.,  300.,  600.,  900., 1200., 1500., 1800., 2100., 2400.,
        2700., 3000.]),
 array([-1.  , -0.88, -0.75, -0.62, -0.5 , -0.38, -0.25, -0.12,  0.  ,
         0.12,  0.25,  0.38,  0.5 ,  0.62,  0.75,  0.88,  1.  ]))

## Perform the Optimisation

### Basin Hopping

In [47]:
class MyBounds:
    """
    Class to constrain the trail correlations of Basin Hopping
    """
    def __init__(self, xmax=[1]*6, xmin=[-1]*6):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))

        return tmax and tmin

In [ ]:
results_basinhopping = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])

now = datetime.now()
now = now.strftime("%d-%m-%H-%M")
FNAME = f'results-{now}.txt'
fout = open(FNAME,'w')

for temp in tqdm(np.linspace(0, 10000, num=11)):
    for x in [0]:#tqdm(np.linspace(-1,1,9)):
        clear_output(wait=True)
        fout.write(f"ECI {eci}\n")
        fout.write(f"Optimising for T: {temp}, 1-pt corr: {x}\n")
        FIXED_CORR_1 = x

        rho_pair = np.array([None,None])
        
        #Make a first guess
        corrs0 = np.array([1, *np.random.uniform(-1, 1, 5)])
        linear_constraints = []

        #Linear Constraint for each rho to be between 0 and 1
        for cluster_idx, _ in clusters.items():
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        #Set bounds for the local minimisation 
        #limit correlations to [1, 1-pt, [-1,1], [-1,1], ...]
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)]
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 5000,
                   'xtol'    : 1e-15,
                   'initial_constr_penalty' : 10,
                  }
        
        minimizer_kwargs = {'args':(vmat, kb, clusters, configs, configcoef,temp,eci),
                            'method': 'trust-constr',
                            'options': options,
                            'jac': F_jacobian, 'hess': F_hessian,
                            'constraints' : [{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,]},
                                             *linear_constraints, 
                                             {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                             {'fun': constraint_zero, 'type':'eq',},
                                            ],
                            'bounds': bounds_corrs,
                           }
        
        mybounds = MyBounds(xmax=[1, FIXED_CORR_1,*[1]*(len(clusters)-2)], xmin=[1, FIXED_CORR_1,*[-1]*(len(clusters)-2)])
        
        res = basinhopping(F, 
                           corrs0, #first guess
                           niter=100, #total num of iterations
                           T=1.0, #temp for Metropolis MC trial search
                           stepsize=10,
                           minimizer_kwargs=minimizer_kwargs,
                           niter_success=20, #num iters to exit after no new minima found 
                           interval=5, #num iters to change step size
                           disp=True,
                           accept_test=mybounds,
                           #callback=basin_hopping_callback
                          )
        
        fout.write(f'1-pt Correlation: {res.x[1]:.2f}\n')
        fout.write(f'Concentation: {(res.x[1] - (-1))/(1 - (-1)):.2f}\n')
        fout.write(f'New Minima found --> G: {res.fun:.2f}\n')
        fout.write(f'Correlations: {res.x}\n')
        fout.write('Current Rho:\n')
        for cluster_idx in clusters:
            fout.write(f'{np.matmul(vmat[cluster_idx],res.x)}\n')
        fout.write("===========================\n")
        fout.flush()
        os.fsync(fout.fileno())

        #Code to extract rho for pair for sanity check. Not used.
        for cluster_idx in clusters:
            if cluster_idx == 2:
                rho_pair = np.matmul(vmat[cluster_idx],res.x)
                break
        
        results_basinhopping = results_basinhopping.append({'T' : temp, 
                                                     '1-point_corr' : x, 
                                                     'F' : res.fun, 
                                                     'corrs': res.x,
                                                     'rhos': rho_pair
                                                    }, 
                                                    ignore_index = True
                                                   )

#save results
results_basinhopping.to_csv(f'phaseDiag_{now}.csv',index=False)

In [ ]:
fig = go.Figure()

for T in results_basinhopping['T'].unique():
    fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['T'] == T]['1-point_corr'],
                             y = results_basinhopping[results_basinhopping['T'] == T]['F'],
                             mode='markers+lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title=f"F vs 1-point Corr - {eci}",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

In [ ]:
fig = go.Figure()
#spec = 
for T in results_basinhopping['T'].unique():
    fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['T'],
                             y = np.stack(results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['corrs'].values)[:,2],
                             mode='markers+lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title=f"T vs 2-point Corr: {eci}",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

### Uniform Sampling

In [ ]:
results_uniform = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])
NUM_TRIALS = 500

for temp in [3000]:#tqdm(np.linspace(0, 3000, num=11)):
    for x in tqdm(np.linspace(-1,1,17)):
        FIXED_CORR_1 = x
        
        MIN_RES_VAL = 1e5 #random large number
        rho_pair = np.array([None,None])
        #corrs0 = np.array([1, *np.random.uniform(-1, 1, 5)])
        corrs0 = np.array([1, *np.random.uniform(-1, 1, 2)])
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)]
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 5000,
                   'xtol'    : 1e-15,
                   'initial_constr_penalty' : 10,
                  }
        
        for _ in tqdm(range(NUM_TRIALS)):
            
            res = minimize(F,
                           corrs0,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci),
                           options=options,
                           jac=F_jacobian, hess=F_hessian,
                           constraints=[{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,],'hess': 0},
                                        *linear_constraints, 
                                        {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                        {'fun': constraint_zero, 'type':'eq',},
                                       ],
                           bounds=bounds_corrs,
                          )
            
            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for x:{x}, T:{temp} fun: {MIN_RES_VAL}")
        
            
        for cluster_idx in clusters:
            if cluster_idx == 2:
                rho_pair = np.matmul(vmat[cluster_idx],MIN_RES.x)
                break
        
        results_uniform = results_uniform.append({'T' : temp, 
                                           '1-point_corr' : x, 
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                           'rhos': rho_pair
                                          }, 
                                          ignore_index = True
                                         )
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
results_uniform.to_csv(f'phaseDiag_{now}_uniform.csv',index=False)

In [ ]:
fig = go.Figure()

for T in results_uniform['T'].unique():
    fig.add_trace(go.Scatter(x = results_uniform[results_uniform['T'] == T]['1-point_corr'],
                             y = results_uniform[results_uniform['T'] == T]['F'],
                             mode='markers+lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()